### Learning convergence check in Fusion ART based on minimum delta threshold 

Let define a fusion ART network with three input channels

In [ ]:
from fusionART import *

#fa is defined as FusionART with schema based input fields definition. For example, here it consists of 'state', 'action', and 'Q'
fa = FusionART(schema=[{'name':'state','compl':True,'attrib':['s1','s2']},{'name':'action','attrib':['a1','a2','a3','a4']},{'name':'Q','compl':True,'attrib':['q']}],beta=[1.0,1.0,1.0],alpha=[0.1,0.1,0.1],gamma=[1.0,1.0,1.0],rho=[0.0,0.0,0.2])

#the structure of the fusionART network can be shown with the methods as follows
fa.displayNetwork()
fa.F1Fields

A new wrapper function can be defined to conduct the fusionART learning and check if there's any change made by the learning so to conclude that the learning has been converged 

In [ ]:
## a new function of automated learning for fusion ART (fusart) object that returns a boolean True
# if there's any change to the weight values larger than mindelta or a recruitment of new uncommitted code for node J
# Otherwise it returns False.
# The function accept fusart argument as the fusion ART object, J as the selected F2 node from resonance search, and mindelta to be the minimum change of the weight to consider as converged
def xAutoLearn(fusart=None, J=None, mindelta=0.00001):
    newcode = fusart.uncommitted(J)
    bw = copy.deepcopy(fusart.codes[J]['weights']) #keep the weights values before of J before learning
    fusart.autoLearn(J)
    if not newcode:
        return any([any([abs(bw[k][i]-fusart.codes[J]['weights'][k][i]) > mindelta for i in range(len(bw[k]))]) for k in range(len(bw))])
    else:
        return True


### Example of the use of xAutoLearn function

The xAutoLearn function can be used at one time to indicate if weights has been changed after the learning

In [ ]:
#present the input to fusionART object fa
fa.updateF1bySchema([{'name':'state', 'val':[0.0,1.0]},{'name':'action', 'val':[1.0,0,0,0]},{'name':'Q','val':[0.3]}])
print("set F1 by schema ", [{'name':'state', 'val':[0.0,1.0]},{'name':'action', 'val':[1.0,0,0,0]},{'name':'Q','val':[0.3]}])

#perform resonance search to select F2 node J
print("resonance search: ")
J = fa.resSearch()
print("selected ", J)

#perform learning and return the status for any change
changed = xAutoLearn(fa, J)
fa.displayNetwork()

#print out the status of any change. Running this cell again consecutively will show different status of change
if changed:
    print('there is a change in weights')
else:
    print('no change in the weights')

For a dataset containing several data items, the xAutoLearn function can be used to count the number of change so that a zero count means the learning has converged 

In [ ]:
#an example data set in a list of F1 schema
episode_data_2 = [ [{'name':'state', 'val':[1.0,1.0]},{'name':'action', 'val':[1.0,0,0,0]},{'name':'Q','val':[0.3]}],
                [{'name':'state', 'val':[0.0,1.0]},{'name':'action', 'val':[0.0,1.0,0,0]},{'name':'Q','val':[0.5]}],
                [{'name':'state', 'val':[0.0,0.0]},{'name':'action', 'val':[1.0,0,0,1.0]},{'name':'Q','val':[0.7]}],
                [{'name':'state', 'val':[1.0,0.0]},{'name':'action', 'val':[0.0,1.0,0,0]},{'name':'Q','val':[0.5]}],
                [{'name':'state', 'val':[0.0,1.0]},{'name':'action', 'val':[1.0,0,0,0]},{'name':'Q','val':[0.3]}] ]

#another example data set in a list of F1 schema
episode_data_3 = [ [{'name':'state', 'val':[0.0,0.0]},{'name':'action', 'val':[1.0,0,0,1.0]},{'name':'Q','val':[0.5]}],
                [{'name':'state', 'val':[1.0,0.0]},{'name':'action', 'val':[0.0,0,0,1.0]},{'name':'Q','val':[0.7]}],
                [{'name':'state', 'val':[1.0,0.0]},{'name':'action', 'val':[0.0,0,1.0,0]},{'name':'Q','val':[0.3]}],
                [{'name':'state', 'val':[0.0,0.0]},{'name':'action', 'val':[1.0,0,0,0]},{'name':'Q','val':[0.4]}] ]

#a function that receive an episode data to learn it in fusion ART
def episode_learn(edata=None):
    cnt = 0
    for d in edata:
        fa.updateF1bySchema(d)
        J = fa.resSearch()
        cnt += xAutoLearn(fa, J) #the status is counted 
    return cnt

print('learning (episode 2)')
print('number of new data learned (episode 2): ', episode_learn(edata=episode_data_2))
fa.displayNetwork()
print("")
print('learning (episode 3)')
print('number of new data learned (episode 3): ', episode_learn(edata=episode_data_3))
fa.displayNetwork()
print("")

print('learning (episode 2)')
print('number of new data learned (episode 2): ', episode_learn(edata=episode_data_2))
fa.displayNetwork()
print("")
print('learning (episode 3)')
print('number of new data learned (episode 3): ', episode_learn(edata=episode_data_3))
fa.displayNetwork()

##Running this cell multiple times would eventually make the number of data learned = 0
##which means the learning has converged

# ART2 Operation

### I. Choice Function
 ART2: $ T_j = \sum_{k=1}^n \gamma^{k} \frac{{\bf x}^{k} \cdot {\bf w}_j^{k}}{||{\bf x}^{k}||\;||{\bf w}_j^{k}||} $, the original FuzzyART Choice function $T_j = \sum_{k=1}^3 \gamma^{k} \frac{|{\bf x}^{k} \wedge {\bf w}_j^{k}|}{\alpha^{k}+|{\bf w}_j^{k}|}$

In [ ]:
#choiceFieldFuncART2: calculate the choice function for a node j in field k
#given input xck, a weight vector wjck, alphack, and gammack for contribution parameter and choice parameters 
#return the activation value of node j for particular field k
#The template for ART2 choice function for a particular channel/field can be defined as follows:
def choiceFieldFuncART2(xck,wjck,alphack,gammack):
    tp = np.dot(np.array(xck),np.array(wjck))
    btm = np.linalg.norm(np.array(xck))*np.linalg.norm(np.array(wjck))
    return gammack * (float(tp)/float(btm))

### II. Code Competition
$T_J = \max \{T_j: \mbox{for all $F_2$ node $j$}\}$  (the same as the original code competition in FusionART)

### III. Template Matching
ART2: $ m_J^{k} = \frac{{\bf x}^{k} \cdot {\bf w}_j^{k}}{||{\bf x}^{k}||\;||{\bf w}_j^{k}||} \geq \rho_s^{k} $, the original FuzzyART template matching: $m_J^{k} = \frac{|{\bf x}^{k} \wedge {\bf w}_J^{k}|}{|{\bf x}^{k}|} \geq \rho_s^{k}$

In [ ]:
#matchFuncART2: ART2 match function of weight vector wjck with vector xck
#return the match value. 
##The template function for ART2 template matching for a particular channel/field can defined as follows: 
def matchFuncART2(xck,wjck):
    m = 0.0
    denominator = 0.0
    tp = np.dot(np.array(xck),np.array(wjck))
    btm = np.linalg.norm(np.array(xck))*np.linalg.norm(np.array(wjck))
    if btm <= 0:
        return 1.0
    return float(tp)/float(btm)

#The template include the checking of the match with vigilance parameter
def resonanceFieldART2(xck, wjck, rhok):
    return matchFuncART2(xck, wjck) < rhok

### IV. Template Learning
ART2: ${\bf w}_J^{k {\rm (new)}} = (1-\beta^{k}) {\bf w}_J^{k {\rm (old)}} + \beta^{k} {\bf x}^{k}$, the original FuzzyART template learning ${\bf w}_J^{k {\rm (new)}} = (1-\beta^{k}) {\bf w}_J^{k {\rm	(old)}} + \beta^{k} ({\bf x}^{k} \wedge {\bf w}_J^{k {\rm (old)}})$


In [ ]:
#updWeightsART2: ART template learning function of weight vector wjck with vector xck
#return the updated weight. 
##The template function for ART2 template learning for a particular channel/field can defined as follows: 
def updWeightsART2(rate, weightk, inputk):
    w = np.array(weightk)
    i = np.array(inputk)
    uw = (((1-rate)*w) + (rate*i)).tolist()
    return uw

### L2 Norm and ART2 Complement Coding
The L2 Norm used in ART2 can be defined as follows $||{\bf p}||\equiv\sqrt{\sum_ip^2_i}$

Complement coding in ART2 is based on euclidean distance $\bar{I}_i^{k}=\sqrt{1-(I_i^{k})^2}$ instead of just the normalized inverse from 1 like $\bar{I}_i^{k}=1-I_i^{k}$

In [ ]:
#Given a vector (list) I in Python
I = [1, 0, 0.3, 0.5, 1, 0]
#complement coded vector Ic of I based on euclidean distance for ART2 can be made as follows
Ic = I + [math.sqrt(1-i**2) for i in I]
Ic

A FusionART object can be made to use ART2 operations instead of Fuzzy ART by re-defining the ART2 template functions to the FusionART object.  

In [ ]:
#Let define another fusionART object fa2 with identical structure as before
fa2 = FusionART(schema=[{'name':'state','compl':True,'attrib':['s1','s2']},{'name':'action','attrib':['a1','a2','a3','a4']},{'name':'Q','compl':True,'attrib':['q']}],beta=[1.0,1.0,1.0],alpha=[0.1,0.1,0.1],gamma=[1.0,1.0,1.0],rho=[0.0,0.0,0.2])

#The original FuzzyART methods for resonance search can be overriden with ART2 template functions 
#defined above as follows

#set the choice function to activate category field
fa2.setChoiceActFunction(cfunction=choiceFieldFuncART2)

#set the weight update function
fa2.setUpdWeightFunction(ufunction=updWeightsART2)

#set the resonance search function
fa2.setResonanceFieldFunction(rfunction=resonanceFieldART2)

#set the match function
fa2.setMatchValFieldFunction(mfunction=matchFuncART2)


Now the fusionART object (fa2) will apply ART2 for all operations in the cycles of resonance search and learning.
However, since the fusionART is defined to be schema-based, the automated complement coding function is still using the original subtraction scheme as used in FuzzyART.

To make the fusionART object applies the euclidean-distance based complement coding, the function to handle the complement coding in the schema can be overridden.

In [ ]:
#Let define the function for refreshing/updating the schema with the new euclidean method of complement coding
def refreshComplSchema(fschema):
    uschema = copy.deepcopy(fschema)
    if 'compl' in uschema:
        if uschema['compl']:
            uschema['vcompl'] = [math.sqrt(1-i**2) for i in uschema['val']] #the euclidean complement
    return uschema

#based on the new function definition, the corresponding function in fusionART can be overridden
#refreshComplSchema = refreshComplEuclidSchema
fa2.setUpdRefComplFunction(rcfunction=refreshComplSchema)


The new fusion ART object with ART2 operations is ready to test

In [ ]:
#present the input to fusionART object fa
fa2.updateF1bySchema([{'name':'state', 'val':[0.0,1.0]},{'name':'action', 'val':[1.0,0,0,0]},{'name':'Q','val':[0.3]}])
print("set F1 by schema ", [{'name':'state', 'val':[0.0,1.0]},{'name':'action', 'val':[1.0,0,0,0]},{'name':'Q','val':[0.3]}])

#perform resonance search to select F2 node J
print("resonance search: ")
J = fa2.resSearch()
print("selected ", J)

#perform learning and return the status for any change
changed = xAutoLearn(fa2, J)
fa2.displayNetwork()


### Hybrid FuzzyART and ART2

ART2 can be defined to apply in a field side by side with another field together in one FusionART object.
For example, we can define EM-ART network wherein the top fusion ART is defined to have two fields one with ART2 and the other is FuzzyART

In [ ]:
#Let define EM-ART model as two fusion ARTs stacked on top of another
#The top fusionART has two input fields: 'events' to store the episode sequence, and 'winning' for the winning status
#The 'events' field will be applied with ART2 while 'winning' field stays with the default FuzzyART
ftop = FusionART(schema=[{'name':'events','compl':True,'attrib':['y1']},{'name':'winning','compl':True,'attrib':['win']}],beta=[1.0,1.0],alpha=[0.1,0.1],gamma=[1.0,1.0],rho=[1.0,1.0],numarray=True)
fbottom = FusionART(schema=[{'name':'state','compl':True,'attrib':['s1','s2']},{'name':'action','attrib':['a1','a2','a3','a4']},{'name':'Q','compl':True,'attrib':['q']}],beta=[1.0,1.0,1.0],alpha=[0.1,0.1,0.1],gamma=[1.0,1.0,1.0],rho=[1.0,1.0,1.0],numarray=True)

fbottom.stackTopFusionART(ftop)
fbottom.linkF2TopF1BySchema(['events'])

ftop.displayNetwork()
print("===========================")
fbottom.displayNetwork()

All the necessary operations of ART2 can be made to override the pre-existing FuzzyART methods for the particular field in the top Fusion ART 

In [ ]:
#The default functions of FuzzyART for a particular field can be overridden by specifying the field
#index k. k=0 arguments refers to apply ART2 only for the first field of the top fusion ART 

#set the choice function to activate category field
ftop.setChoiceActFunction(cfunction=choiceFieldFuncART2, k=0)

#set the weight update function
ftop.setUpdWeightFunction(ufunction=updWeightsART2, k=0)

#set the resonance search function
ftop.setResonanceFieldFunction(rfunction=resonanceFieldART2, k=0)

#set the match function
ftop.setMatchValFieldFunction(mfunction=matchFuncART2, k=0)

#set the refresh complement schema function
ftop.setUpdRefComplFunction(rcfunction=refreshComplSchema, k=0)

In [ ]:
#Some testing of the new hybrid model of EM-ART

print("")
fbottom.updateF1bySchema([{'name':'state', 'val':[0.0,1.0]},{'name':'action', 'val':[1.0,0,0,0]},{'name':'Q','val':[0.3]}])
print("set F1 by schema ", [{'name':'state', 'val':[0.0,1.0]},{'name':'action', 'val':[1.0,0,0,0]},{'name':'Q','val':[0.3]}])
print("resonance search: ")
J = fbottom.SchemaBasedSequentialResSearch()
print("selected ", J)
if fbottom.uncommitted(J):
	print('uncommitted')
fbottom.autoLearn(J)
ftop.displayNetwork()
fbottom.displayNetwork()

print("")
fbottom.updateF1bySchema([{'name':'state', 'val':[1.0,0.0]},{'name':'action', 'val':[1.0,0,0,1.0]},{'name':'Q','val':[0.6]}])
print("set F1 by schema ", [{'name':'state', 'val':[1.0,0.0]},{'name':'action', 'val':[0,0,0,1.0]},{'name':'Q','val':[0.6]}])
print("resonance search: ")
J = fbottom.SchemaBasedSequentialResSearch()
print("selected ", J)
if fbottom.uncommitted(J):
	print('uncommitted')
fbottom.autoLearn(J)
ftop.displayNetwork()
fbottom.displayNetwork()

print("")
fbottom.updateF1bySchema([{'name':'state', 'val':[0.0,1.0]},{'name':'action', 'val':[1.0,0,0,0]},{'name':'Q','val':[0.3]}])
print("set F1 by schema ", [{'name':'state', 'val':[0.0,1.0]},{'name':'action', 'val':[1.0,0,0,0]},{'name':'Q','val':[0.3]}])
print("resonance search: ")
J = fbottom.SchemaBasedSequentialResSearch()
print("selected ", J)
if fbottom.uncommitted(J):
	print('uncommitted')
fbottom.autoLearn(J)
ftop.displayNetwork()
fbottom.displayNetwork()

print("")
print('set the winning status')
ftop.updateF1bySchema([{'name':'winning','val':[1.0]}])
ftop.displayNetwork()
fbottom.displayNetwork()

print("")
print("learn the episode")
J=ftop.resSearch()
ftop.autoLearn(J)
ftop.displayNetwork()
ftop.clearActivityF1()
